# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |   Julieta Madrigal Flores|
| **Fecha**      |   Lunes 16 de febrero 2026|
| **Expediente** |   744029|

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

**R =** para poder divir nuestro dataset en la parte de entrenamiento y la parte de Test, esto nos permite identificar si nuestro modelo esta generaliza correctamente o se sobreajusta.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Realmente si porque de esa manera tendrías todo y no habría margen de error pero lo que tienes que hacer es dividir toda tu información en train y test, comparar su R2 para verificiar que el modelo de entrenamiento sea capaz de generalizar correctamente y una vez que lo confirmes, los juntas y haces una función completa.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
data = pd.read_excel('Motor Trend Car Road Tests.xlsx')
data.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [6]:
y = data['mpg']

In [7]:
X = data.drop(['mpg', 'model'], axis=1)

In [22]:
np.reshape(X_valid, [1, -1])

array([[  6.  , 160.  , 110.  ,   3.9 ,   2.62,  16.46,   0.  ,   1.  ,
          4.  ,   4.  ]])

In [26]:
# definimos que n es la cantidad de filas que tiene la matriz de X
n = len(X)
# guarda los errores cuadraticos  (y_valid - y_pred)**2
errores = []

for i in range(n):

    #  X.iloc[i, :] = Dame la fila i completa (todas sus variables)
    X_valid = X.iloc[i, :]
    # me da el valor real asociado a esa observación.
    y_valid = y[i]

  # X[:i] = Toma todas las filas antes de la posición i
  # X[i+1:] = Toma todas las filas después de la posición
    X_train = np.concatenate((X[:i], X[i+1:]), axis=0)
    y_train = np.concatenate((y[:i], y[i+1:]), axis=0)

# Hago mi regresión con la parte de entrenamiento
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)

# Hacemos la predicción de Y
    y_pred = modelo.predict(np.reshape(X_valid, [1, -1]))

    # usamos mean_squared_error normal
    error = mean_squared_error([y_valid], y_pred)
    errores.append(error)

print("MSE promedio :", np.mean(errores))
print("Desviación estándar:", np.std(errores))



MSE promedio : 12.181558006901943
Desviación estándar: 17.067399871888522


In [28]:
# El error cuadrático fuera de muestra de cada observación
errores

[5.258855796629032,
 2.453996713086818,
 20.55049341688269,
 0.04432910713964311,
 1.5811555159760267,
 10.118625488875358,
 0.016374041548474508,
 8.081250710389499,
 39.4701834796497,
 0.7706327722901085,
 4.955694234215848,
 10.224730198354443,
 4.430129331402683,
 0.4878127712478519,
 6.823844575809427,
 0.6027506975546951,
 36.8101786099587,
 31.759855581210353,
 1.063227541805789,
 32.713286187027215,
 14.30717368271277,
 3.4053546499989142,
 9.408007649079861,
 0.0001035003980364952,
 9.962439534547674,
 1.3412208919431396,
 0.16480064082222703,
 23.60020650224925,
 83.1145764338518,
 0.00010271237184881921,
 8.785533367966924,
 17.502929883865395]

Interpreta.

Después de hacer una validación de *Leave-One-Out*, el modelo tiene un MSE ( mean squared error ) promedio de 12.18 y una desviación estándar de 17.07, esto significa que tiene una alta variabilidad de error individual; es decir, que los errores están muy dispersos alrededor de la media.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3